In [1]:
!pip install folium pandas

  Using cached folium-0.19.7-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached branca-0.8.1-py3-none-any.whl.metadata (1.5 kB)
Using cached folium-0.19.7-py2.py3-none-any.whl (112 kB)
Using cached branca-0.8.1-py3-none-any.whl (26 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [folium]


In [2]:
import boto3
import pandas as pd
import io
from botocore.exceptions import ClientError

In [3]:
def read_csv_from_s3_with_options(bucket_name, file_key, **kwargs):
    """
    Enhanced version with additional pandas read_csv options
    
    Parameters:
    bucket_name (str): Name of the S3 bucket
    file_key (str): Path to the CSV file in the bucket
    **kwargs: Additional arguments to pass to pd.read_csv()
    
    Returns:
    pandas.DataFrame: DataFrame containing the CSV data
    """
    try:
        # Create an S3 client
        s3_client = boto3.client('s3')
        
        # Get the object from S3
        response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
        
        # Read the CSV content with additional options
        df = pd.read_csv(io.BytesIO(response['Body'].read()), **kwargs)
        
        return df
    
    except Exception as e:
        print(f"Error reading CSV file from S3: {str(e)}")
        raise



In [4]:
## reading training data
abs
if __name__ == "__main__":
    try:
        bucket_name = "birdsbucker"
        file_key = "trainingdata/training_data.csv"
        
        # Example with additional pandas read_csv options
        df = read_csv_from_s3_with_options(
            bucket_name,
            file_key,
            delimiter=',',
            encoding='utf-8',
            na_values=['NA', 'missing']
        )
        
        print("DataFrame shape:", df.shape)
        print("\nFirst few rows:")
        print(df.head())
        
    except Exception as e:
        print(f"Error in main: {str(e)}")

DataFrame shape: (24, 10)

First few rows:
  Segment      Lat     Long  Year  Daily Traffic Volume  % cars  % trucks  \
0      DC  38.9072 -77.0369  2019                200000      40        15   
1      DC  38.9072 -77.0369  2020                150000      45        10   
2      DC  38.9072 -77.0369  2021                170000      42        15   
3      DC  38.9072 -77.0369  2022                190000      38        15   
4      DC  38.9072 -77.0369  2023                204000      36        15   

   % suvs   Noise level (db)  total_birds  
0       45         75.560300       2776.0  
1       45         73.910913       1376.0  
2       43         74.794489       3576.0  
3       47         75.397536       1266.0  
4       49         75.766302       2141.0  


In [5]:
def scale_radius(bird_count, min_birds, max_birds):
    """Scale the radius of circles based on bird count"""
    min_radius = 2
    max_radius = 20
    return min_radius + (max_radius - min_radius) * ((bird_count - min_birds) / (max_birds - min_birds))

def get_color(bird_count):
    """Return color based on bird count ranges"""
    # Calculate quartiles for better distribution
    q1 = df['total_birds'].quantile(0.25)
    q2 = df['total_birds'].quantile(0.5)
    q3 = df['total_birds'].quantile(0.75)
    
    if bird_count <= q1:
        return '#d7191c'  # Red'
    elif bird_count <= q2:
        return 'fdae61'  # Orange
    elif bird_count <= q3:
        return '2b83ba'  # Light blue'
    else:
        return '#abdda4'  # Light green

In [6]:
def add_df_point_to_map(df, m):
    """
    Add points from DataFrame to map with size and color based on total_birds
    
    Parameters:
    df: DataFrame containing latitude, longitude, and total_birds
    m: folium Map object
    
    Returns:
    feature_group: Folium FeatureGroup containing all markers
    """

    min_birds = df['total_birds'].min()
    max_birds = df['total_birds'].max()
    
    # Create a feature group for the data points
    feature_group = folium.FeatureGroup(name='Bird Observations')
    
    # Add points to feature group
    for idx, row in df.iterrows():
        folium.CircleMarker(
            location=[row['Lat'], row['Long']],
            radius=scale_radius(row['total_birds'], min_birds, max_birds),
            color=get_color(row['total_birds']),
            fill=True,
            fill_color=get_color(row['total_birds']),
            fill_opacity=0.7,
            popup=f"Total Birds: {row['total_birds']}",
            tooltip=f"Location: ({row['Lat']:.2f}, {row['Long']:.2f})\nTotal Birds: {row['total_birds']}"
        ).add_to(feature_group)
    
    # Add the feature group to the map
    feature_group.add_to(m)
    
    # Add a legend
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; right: 50px; 
                border:2px solid grey; z-index:9999; font-size:14px;
                background-color: white;
                padding: 10px;
                border-radius: 5px;
                ">
    <p><b>Bird Count Legend</b></p>
    <p>
    <i class="fa fa-circle" style="color:blue"></i> Low<br>
    <i class="fa fa-circle" style="color:yellow"></i> Medium<br>
    <i class="fa fa-circle" style="color:orange"></i> High<br>
    <i class="fa fa-circle" style="color:red"></i> Very High
    </p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Return the feature group
    return feature_group

In [20]:
import folium
from folium import plugins
import numpy as np

# Create a map centered roughly on the middle of I-95
m = folium.Map(location=[40.0, -75.0], zoom_start=5)

# Detailed I-95 coordinates (more waypoints for accurate route)
i95_detailed = [
    [25.7617, -80.1918],  # Miami, FL
    [25.9860, -80.1626],  # North Miami
    [26.1224, -80.1373],  # Fort Lauderdale
    [26.7153, -80.0534],  # West Palm Beach
    [27.3364, -80.2309],  # Fort Pierce
    [28.0345, -80.6881],  # Melbourne
    [28.5383, -81.3792],  # Orlando
    [29.2108, -81.0228],  # Daytona Beach
    [29.6516, -81.3279],  # Palatka
    [30.3322, -81.6557],  # Jacksonville
    [30.8268, -81.9598],  # Brunswick, GA
    [31.1499, -81.4914],  # Jekyll Island
    [31.9976, -81.1130],  # Savannah
    [32.2163, -81.0976],  # Port Wentworth
    [32.4829, -81.2332],  # Statesboro
    [33.5007, -81.9574],  # Augusta
    [34.0007, -81.0348],  # Columbia, SC
    [34.2257, -80.6890],  # Camden
    [34.6235, -79.9414],  # Bennettsville
    [34.7554, -79.4628],  # Lumberton, NC
    [35.1540, -78.8986],  # Fayetteville
    [35.7796, -78.6382],  # Raleigh
    [36.0956, -78.3097],  # Rocky Mount
    [36.4615, -77.6527],  # Roanoke Rapids
    [36.8529, -76.2856],  # Norfolk, VA
    [37.5407, -77.4360],  # Richmond
    [38.1854, -77.5514],  # Fredericksburg
    [38.9072, -77.0369],  # Washington, DC
    [39.2904, -76.6122],  # Baltimore, MD
    [39.5695, -75.9375],  # Newark, DE
    [39.9526, -75.1652],  # Philadelphia, PA
    [40.2206, -74.7597],  # Trenton, NJ
    [40.7128, -74.0060],  # New York City, NY
    [40.9168, -73.7832],  # New Rochelle
    [41.1856, -73.1952],  # Bridgeport, CT
    [41.3083, -72.9279],  # New Haven
    [41.5232, -72.7009],  # Meriden
    [41.7658, -72.6734],  # Hartford
    [41.8240, -71.4128],  # Providence, RI
    [42.0654, -71.2478],  # Attleboro, MA
    [42.3601, -71.0589],  # Boston
    [42.7654, -70.9292],  # Newburyport
    [43.0718, -70.7626],  # Portsmouth, NH
    [43.4971, -70.4134],  # Kennebunk, ME
    [43.6591, -70.2568],  # Portland
    [44.1002, -69.1092],  # Rockland
    [44.8023, -68.7778],  # Bangor
    [45.1373, -67.7813],  # Maine Border
]

# Create a feature group for the detailed route
route_group = folium.FeatureGroup(name='I-95 Detailed Route')

# Draw I-95 as a line with the detailed coordinates
folium.PolyLine(
    i95_detailed,
    weight=3,
    color='blue',
    opacity=0.8,
    popup='Interstate 95'
).add_to(route_group)

# Create a feature group for waypoints
waypoints_group = folium.FeatureGroup(name='I-95 Waypoints')

# # Create a feature group for bird observations
# birds_group = folium.FeatureGroup(name='Bird Observations')

# # Calculate size scaling for circles
# min_birds = df['total_birds'].min()
# max_birds = df['total_birds'].max()

# Add markers for waypoints
for i, coord in enumerate(i95_detailed):
    # Create custom HTML popup content
    popup_content = f"""
    <div style="width: 200px">
        <h4>I-95 Waypoint {i+1}</h4>
        <p>Coordinates: {coord[0]:.4f}, {coord[1]:.4f}</p>
    </div>
    """
    
    # Add marker with custom popup
    folium.CircleMarker(
        location=coord,
        radius=4,
        color='red',
        fill=True,
        fillColor='red',
        fillOpacity=0.7,
        popup=folium.Popup(popup_content, max_width=300)
    ).add_to(waypoints_group)

# Add the feature groups to the map
route_group.add_to(m)
waypoints_group.add_to(m)

stations_layer=create_point_layer(stations, lat_col="latitude", lon_col="longitude", value_col="station_id", layer_name="Stations", color_scale=False)
bird_layer = add_df_point_to_map(df, m)

# Add additional map features
folium.TileLayer('cartodbpositron', name='Light Map').add_to(m)
folium.TileLayer('cartodbdark_matter', name='Dark Map').add_to(m)
folium.TileLayer(
    'Stamen Terrain',
    name='Terrain Map',
    attr='Map tiles by <a href="http://stamen.com">Stamen Design</a>, under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>. Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, under <a href="http://www.openstreetmap.org/copyright">ODbL</a>.'
).add_to(m)

# Add map controls
stations_layer.add_to(m)
bird_layer.add_to(m)
plugins.Fullscreen().add_to(m)
plugins.MousePosition().add_to(m)
plugins.MeasureControl().add_to(m)
folium.LayerControl().add_to(m)

# Add a mini map
minimap = plugins.MiniMap()
m.add_child(minimap)

# Add a title
title_html = '''
<div style="position: fixed; 
    top: 10px; left: 50px; width: 200px; height: 50px; 
    background-color: white; border:2px solid grey; z-index:9999; 
    font-size:16px; padding: 10px">
    <b>Interstate 95 Detailed Route Map</b>
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Save the map
m.save('i95_detailed_route_map.html')

In [8]:
# reading noise data

if __name__ == "__main__":
    try:
        bucket_name = "birdsbucker"
        file_key = "trafficVolume/2023_Traffic_Volume_DC.csv"
        
        # Example with additional pandas read_csv options
        noise = read_csv_from_s3_with_options(
            bucket_name,
            file_key,
            delimiter=',',
            encoding='utf-8',
            na_values=['NA', 'missing']
        )
        
        print("DataFrame shape:", noise.shape)
        print("\nFirst few rows:")
        print(noise.head())
        
    except Exception as e:
        print(f"Error in main: {str(e)}")

DataFrame shape: (36, 9)

First few rows:
  Segment      Lat     Long  Year  Daily Traffic Volume  % cars  % trucks  \
0      DC  38.9072 -77.0369  2019                200000      40        15   
1      DC  38.9072 -77.0369  2020                150000      45        10   
2      DC  38.9072 -77.0369  2021                170000      42        15   
3      DC  38.9072 -77.0369  2022                190000      38        15   
4      DC  38.9072 -77.0369  2023                204000      36        15   

   % suvs   Noise level (db)  
0       45         75.560300  
1       45         73.910913  
2       43         74.794489  
3       47         75.397536  
4       49         75.766302  


In [13]:
if __name__ == "__main__":
    try:
        bucket_name = "birdsbucker"
        file_key = "I95_stations_master.csv"
        
        # Example with additional pandas read_csv options
        stations = read_csv_from_s3_with_options(
            bucket_name,
            file_key,
            delimiter=',',
            encoding='utf-8',
            na_values=['NA', 'missing']
        )
        
        print("DataFrame shape:", stations.shape)
        print("\nFirst few rows:")
        print(stations.head())
        
    except Exception as e:
        print(f"Error in main: {str(e)}")

DataFrame shape: (593, 42)

First few rows:
  record_type  state_code station_id  travel_dir  travel_lane  year_record  \
0           S          11     001295           1            0           23   
1           S          11     001295           1            1           23   
2           S          11     001295           1            2           23   
3           S          11     001295           1            3           23   
4           S          11     002295           5            1           23   

  f_system  num_lanes sample_type_volume  num_lanes_volume  ...  county_code  \
0       1U          3                  N                 3  ...            0   
1       1U          3                  T                 3  ...            0   
2       1U          3                  T                 3  ...            0   
3       1U          3                  T                 3  ...            0   
4       1U          3                  T                 3  ...            0   

  is_s

In [14]:
stations.columns

Index(['record_type', 'state_code', 'station_id', 'travel_dir', 'travel_lane',
       'year_record', 'f_system', 'num_lanes', 'sample_type_volume',
       'num_lanes_volume', 'method_volume', 'sample_type_class',
       'num_lanes_class', 'method_class', 'algorithm_volume', 'num_classes',
       'sample_type_truck', 'num_lanes_truck', 'method_truck', 'calibration',
       'data_retrieval', 'type_sensor_1', 'type_sensor_2', 'primary_purpose',
       'lrs_id', 'lrs_point', 'latitude', 'longitude', 'shrp_id',
       'prev_station_id', 'year_established', 'year_discontinued',
       'county_code', 'is_sample', 'sample_id', 'nhs', 'posted_route_signing',
       'posted_signed_route', 'con_route_signing', 'con_signed_route',
       'station_location', 'state'],
      dtype='object')

In [17]:
import folium
import pandas as pd
from typing import Optional, Union, Tuple

def create_point_layer(
    df: pd.DataFrame,
    lat_col: str,
    lon_col: str,
    value_col: str,
    layer_name: str,
    color_scale: bool = True,
    min_value: Optional[float] = None,
    max_value: Optional[float] = None,
    default_color: str = '#3388ff',  # Default folium blue color
    color_range: Tuple[str, str] = ('#ff0000', '#800000')  # Light red to dark red
) -> folium.FeatureGroup:
    """
    Create a layer of points from a DataFrame with optional color intensity based on a numeric value.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing the geographical points and values
    lat_col : str
        Name of the latitude column in the DataFrame
    lon_col : str
        Name of the longitude column in the DataFrame
    value_col : str
        Name of the column containing numeric values for color intensity
    layer_name : str
        Name of the layer for the layer control
    color_scale : bool, optional
        Whether to scale colors based on values (default: True)
    min_value : float, optional
        Minimum value for color scaling. If None, uses DataFrame minimum
    max_value : float, optional
        Maximum value for color scaling. If None, uses DataFrame maximum
    default_color : str, optional
        Color to use when color_scale is False
    color_range : tuple of str, optional
        Two hex colors defining the range for color scaling (start, end)
        
    Returns:
    --------
    folium.FeatureGroup
        Layer containing all points that can be added to any map
    """
    
    # Input validation
    required_columns = [lat_col, lon_col, value_col]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"DataFrame must contain columns: {required_columns}")
    
    # Create feature group
    layer = folium.FeatureGroup(name=layer_name)
    
    # Set min and max values for color scaling if needed
    if color_scale:
        if min_value is None:
            min_value = df[value_col].min()
        if max_value is None:
            max_value = df[value_col].max()
    
    def interpolate_color(normalized_value: float, color1: str, color2: str) -> str:
        """Helper function to interpolate between two hex colors"""
        # Convert hex to RGB
        r1, g1, b1 = int(color1[1:3], 16), int(color1[3:5], 16), int(color1[5:7], 16)
        r2, g2, b2 = int(color2[1:3], 16), int(color2[3:5], 16), int(color2[5:7], 16)
        
        # Interpolate
        r = int(r1 + (r2 - r1) * normalized_value)
        g = int(g1 + (g2 - g1) * normalized_value)
        b = int(b1 + (b2 - b1) * normalized_value)
        
        return f'#{r:02x}{g:02x}{b:02x}'
    
    # Add points to layer
    for idx, row in df.iterrows():
        if color_scale:
            # Calculate color intensity
            normalized_value = (row[value_col] - min_value) / (max_value - min_value)
            normalized_value = max(0, min(1, normalized_value))  # Ensure value is between 0 and 1
            color = interpolate_color(normalized_value, color_range[0], color_range[1])
        else:
            color = default_color
        
        # Create popup content
        popup_content = f"""
            <div style="font-family: Arial, sans-serif;">
                <strong>{value_col}:</strong> {row[value_col]}<br>
                <strong>Latitude:</strong> {row[lat_col]:.4f}<br>
                <strong>Longitude:</strong> {row[lon_col]:.4f}
            </div>
        """
        
        # Add circle marker to layer
        folium.CircleMarker(
            location=[row[lat_col], row[lon_col]],
            radius=8,
            popup=folium.Popup(popup_content, max_width=300),
            tooltip=f"{value_col}: {row[value_col]}",
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            weight=2
        ).add_to(layer)
    
    return layer

In [19]:
stations['longitude'] = -abs(stations['longitude'])